In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import json

In [2]:
DRIVER_PATH = "./driver/chromedriver"

In [3]:
# ranking_data = {}

with open("./colleges_usnews_link_cleaned.json") as f:
    college_usnews_link = json.load(f)

In [4]:
def get_single_college_rankings(college_id, college_name, cur_usnews_link):

    cur_college_ranking_info = {}

    service = Service(executable_path=DRIVER_PATH)
    driver = webdriver.Chrome(service=service)

    try:
        driver.get(cur_usnews_link + "/overall-rankings")
        body = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        try:
            ranking_data_list = body.find_element(By.CLASS_NAME, value="BadgeList-wn7t98-1")
            rank_details = ranking_data_list.find_elements(By.CLASS_NAME, value="eMEqFO")
            for each_badge in rank_details:
                try:
                    each_badge_data = each_badge.find_elements(By.TAG_NAME, value="strong")
                    badge_rank = each_badge_data[0].text[1:]
                    badge_name = each_badge_data[1].text
                    cur_college_ranking_info[badge_name] = badge_rank
                except:
                    continue
        except:
            print(1)
            pass
    except Exception as e:
        print(e)
        pass
    finally:
        driver.quit()
    ranking_data[college_id] = cur_college_ranking_info
    print(college_name)

In [5]:
for key, value in college_usnews_link.items():
    with open("./college_ranking_data.json") as f:
        ranking_data = json.load(f)
    if key not in ranking_data:
        get_single_college_rankings(key, value["college_name"], value["usnews_link"])
        with open("./college_ranking_data.json", "w") as f:
            f.write(json.dumps(ranking_data, indent=4))

Adams State University
Adelphi University
Adrian College
Agnes Scott College
Alabama State University
Alaska Pacific University
Albany College of Pharmacy and Health Sciences
Albany State University
Albertus Magnus College
Albion College
Albright College
Alcorn State University
Alfred University
Alice Lloyd College
Allegheny College
Allen University
Alliant International University-San Diego
Alma College
Alvernia University
Alverno College
American Academy of Art
American International College
American University
American University of Health Sciences
American University of Puerto Rico
Amherst College
Anderson University
Andrews University
Angelo State University
Anna Maria College
Antioch College
Appalachian Bible College
Appalachian State University
Aquinas College
Arcadia University
Arizona Christian University
Arizona College of Nursing-Las Vegas
Arizona State University Campus Immersion
Arkansas Baptist College
Arkansas Tech University
Art Academy of Cincinnati
Art Center College 

In [6]:
with open("./college_ranking_data.json") as f:
    ranking_data = json.load(f)

count = 0
for key, value in college_usnews_link.items():
    if key not in ranking_data:
        print(key)
        count += 1
print(count)

0


## Append data to database

In [7]:
import pymongo

In [8]:
uri = "" # in cledge-db resource, then click Quick Start.
client = pymongo.MongoClient(uri)

db = client["colleges"]
collection = db["colleges-data"]

In [9]:
with open("./college_ranking_data.json") as f:
    ranking_data = json.load(f)

for key, value in ranking_data.items():
    collection.update_one({"UNITID": key}, {"$set": {"us_news_rankings" : value}})